Process TCGA data and save to mongodb

In [2]:
from pymongo import MongoClient
import json
import os

In [10]:
datapath = '/Volumes/originalData/TCGA/2019-02-07_TCGA_masked_CNA'
metafile = '/Volumes/originalData/TCGA/Meta/tcga_masked_cnv_meta_190207.json'
db_collection = 'masked_v2'
with open(metafile, 'r') as fi:
    data = json.load(fi)

In [4]:
hits = data['data']['hits']

In [9]:
samples = []
for file in hits:
    file_path = '{}/{}/{}'.format(datapath,file['id'], file['file_name'])
    
    if file.get('cases')[0].get('diagnoses') == None:
        morphology = None
        tissue_origin = None
        tumor_stage = None
        vital_status = None
        age_at_diagnosis = None
        days_to_death = None
    else:
        morphology = file['cases'][0]['diagnoses'][0]['morphology']
        tissue_origin = file['cases'][0]['diagnoses'][0]['tissue_or_organ_of_origin']
        tumor_stage = file['cases'][0]['diagnoses'][0]['tumor_stage']
        vital_status = file['cases'][0]['diagnoses'][0]['vital_status']
        age_at_diagnosis = file['cases'][0]['diagnoses'][0]['age_at_diagnosis']
        days_to_death = file['cases'][0]['diagnoses'][0]['days_to_death']

    if file.get('cases')[0].get('demographic') == None:
        gender = None
        race = None
        ethnicity = None
        year_of_birth = None
        year_of_death = None
    else:
        gender = file['cases'][0]['demographic']['gender']
        race = file['cases'][0]['demographic']['race']
        ethnicity = file['cases'][0]['demographic']['ethnicity']
        year_of_birth = file['cases'][0]['demographic']['year_of_birth']
        year_of_death = file['cases'][0]['demographic']['year_of_death']    

    
    
    
    
    
    sam = {'file_name':file['file_name'],
            'file_id':file['id'],
            'project_id':file['cases'][0]['project']['project_id'],
            'case_id':file['cases'][0]['case_id'],
            'data_type':file['data_type'],
            'sample_type':file['cases'][0]['samples'][0]['sample_type'],

            'morphology': morphology,
            'tissue_origin': tissue_origin,
            'tumor_stage': tumor_stage,
            'vital_status': vital_status,
            'age_at_diagnosis': age_at_diagnosis,
            'days_to_death': days_to_death,

            'gender': gender,
            'race': race,
            'ethnicity': ethnicity,
            'year_of_birth': year_of_birth,
            'year_of_death': year_of_death,
             'file_path': file_path}
    
    if os.path.isfile(file_path):
        with open(file_path, 'r') as fin:
            next(fin)
            variants_cnv = []
            for line in fin:
                line = line.split()
                chro = str(line[1])
                start = int(float(line[2]))
                end = int(float(line[3]))
                probes = int(line[4])
                value = float(line[5])
                seg = {'chro': chro, 'start':start, 'end':end, 'probes':probes, 'value':value}
                variants_cnv.append(seg)
            if len(variants_cnv) == 0:
                print(file['file_name'])
            sam['variants_cnv'] = variants_cnv
        
    else:
        print(file['file_name'])
    
    samples.append(sam)

In [11]:
db = MongoClient()['tcga']
db[db_collection].drop()
db[db_collection].insert_many(samples)